In [1]:
import pandas as pd
import requests
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import pprint
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
df = pd.read_csv("tracks_per_year.txt",delimiter='<SEP>')
df.to_csv('tracks_per_year.csv')
df.columns

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


Index(['YEAR', 'ID1', 'ArtistName', 'SongTitle'], dtype='object')

In [3]:
del df['ID1']

df.head()

,YEAR,ArtistName,SongTitle
0,1996,Nellie Lutcher,Let Me Love You Tonight
1,1996,Nellie Lutcher,He's A Real Gone Guy
2,1996,Nellie Lutcher,Fine Brown Frame
3,1996,Nellie Lutcher,My New Papa's Got To Have Everything
4,1996,Nellie Lutcher,Fine Brown Frame


In [4]:
df.dtypes

YEAR           int64
ArtistName    object
SongTitle     object
dtype: object

In [5]:
# Get names of indexes for which column year is less than 2008
indexNames = df[ df['YEAR'] < 2008 ].index
 
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)

In [6]:
df.head()

,YEAR,ArtistName,SongTitle
287804,2008,$jammie the Money,Getting warm
287805,2008,00Genesis,Dream Catcher
287806,2008,10 Years,Proud of You
287807,2008,10 Years,Alabama
287808,2008,10 Years,Picture Perfect (In Your Eyes)


In [7]:
df.shape

(75219, 3)

In [8]:
#setting up link to spotify api w spotipy
client_id = "5028c957611248149d8c04007258f254"
client_secret = "aa56a507cb944f488f8835062a94115c"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [24]:
#test dataframe
dft = df.head(10)
dft

,YEAR,ArtistName,SongTitle
287804,2008,$jammie the Money,Getting warm
287805,2008,00Genesis,Dream Catcher
287806,2008,10 Years,Proud of You
287807,2008,10 Years,Alabama
287808,2008,10 Years,Picture Perfect (In Your Eyes)
287809,2008,10 Years,So Long_ Good-bye
287810,2008,10 Years,Beautiful
287811,2008,10 Years,All Your Lies
287812,2008,10 Years,Russian Roulette
287813,2008,10 Years,Actions & Motives


In [23]:
artistname="Lionel Richie"
search_str = "Tonight Will Be Alright"
result= sp.search(search_str+":"+artistname,type="track,artist")

track_id1 = result["tracks"]["items"][0]["id"]
artist_id1 = result['tracks']['items'][0]["album"]['artists'][0]['id']

features_1 = sp.audio_features(track_id1)
features = pd.DataFrame(features_1)

features["followers"] = sp.artist(artist_id1)['followers']['total']
features["tempo"] = sp.audio_analysis(track_id)['track']['tempo']
features["popularity"] = sp.track(idt)["popularity"]
features["Artist"] = artistname
features["Track"] = search_str
features

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,...,tempo,time_signature,track_href,type,uri,valence,followers,popularity,Artist,Track
0,0.0591,https://api.spotify.com/v1/audio-analysis/2Jr1...,0.646,307413,0.84,2Jr12fpsMdIM6weoUDMf9u,0.0141,8,0.0516,-8.213,...,151.072,4,https://api.spotify.com/v1/tracks/2Jr12fpsMdIM...,audio_features,spotify:track:2Jr12fpsMdIM6weoUDMf9u,0.716,1627956,38,Lionel Richie,Tonight Will Be Alright


In [16]:
# iterate over the dataframe row by row
for index_label, row in dft.iterrows():
   # For each row update the 'Bonus' value to it's double  

    try: 
        artistname= row['ArtistName']
        search_str =  row['SongTitle']
        print(artistname+ ","+ search_str )
        result= sp.search(search_str+":"+artistname,type="track,artist")
        print(result["tracks"]["items"][0]["id"])
        
        track_id = result["tracks"]["items"][0]["id"]
        artist_id = result['tracks']['items'][0]["album"]['artists'][0]['id']
        
        features_i = sp.audio_features(track_id)
        features_i = pd.DataFrame(features_i)
        
        #features["genres"] = sp.artist(artist_id)['genres']
        features_i["followers"] = sp.artist(artist_id)['followers']['total']
        features_i["tempo"] = sp.audio_analysis(track_id)['track']['tempo']
        features_i["popularity"] = sp.track(track_id)["popularity"]
        features_i["Artist"] = artistname
        features_i["Track"] = search_str
        features = features.append(features_i)
        
    except: 
        print ('data not found')

$jammie the Money,Getting warm
1boN0BWdCthrH6ZjND3MF6
00Genesis,Dream Catcher
1GVAHPHe229qqje2K1R8Ru
10 Years,Proud of You
1JMGklRieQkiGp7FsYx4n7
10 Years,Alabama
5UGBFGK0cmm5pS9zwXl7H4
10 Years,Picture Perfect (In Your Eyes)
0k6LfLD7vowO9ZE38LfPl3
10 Years,So Long_ Good-bye
data not found
10 Years,Beautiful
6AgtIN7FyBd4zJhZI7mrsz
10 Years,All Your Lies
2Perdw5NdmyHJuSOzXYUTp
10 Years,Russian Roulette
41jfrcIFbuFZe2dQANzmb1
10 Years,Actions & Motives
2QgTdBYfwIDRy892unFoRm


In [20]:
features = features.drop(columns=['analysis_url', 'id', 'track_href', 'type', 'uri'])
features

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,followers,popularity,Artist,Track
0,0.059100,0.646,307413,0.840,0.014100,8,0.0516,-8.213,1,0.0283,151.072,4,0.7160,1627956,38,Lionel Richie,Tonight Will Be Alright
0,0.002410,0.673,270000,0.911,0.686000,7,0.0846,-8.082,1,0.0744,130.006,4,0.3290,8,0,$jammie the Money,Getting warm
0,0.015400,0.772,181107,0.840,0.607000,1,0.1070,-8.262,1,0.3110,93.091,4,0.6570,880039,0,00Genesis,Dream Catcher
0,0.815000,0.319,351587,0.422,0.053700,9,0.2890,-10.800,1,0.0292,108.481,3,0.0407,390624,26,10 Years,Proud of You
0,0.000991,0.330,234333,0.863,0.000252,9,0.5090,-3.109,1,0.0419,175.897,4,0.1820,390624,24,10 Years,Alabama
0,0.033800,0.301,380480,0.595,0.020200,1,0.7960,-5.306,0,0.0397,119.778,4,0.1930,390624,30,10 Years,Picture Perfect (In Your Eyes)
0,0.001690,0.499,195893,0.735,0.000002,6,0.1620,-4.679,1,0.0323,131.911,4,0.2280,390624,49,10 Years,Beautiful
0,0.007480,0.453,226000,0.854,0.000388,11,0.1450,-3.805,1,0.0419,159.955,3,0.2190,390624,25,10 Years,All Your Lies
0,0.002320,0.433,229200,0.874,0.006940,3,0.0971,-4.224,0,0.0674,150.156,1,0.3340,390624,34,10 Years,Russian Roulette
0,0.000049,0.490,203333,0.906,0.000145,6,0.5320,-2.791,0,0.0380,151.072,4,0.5800,390624,32,10 Years,Actions & Motives
